## Monte Carlos Methos e Temporal Difference Method

### Envirorment

* N estados não terminais $S+ = {s1, s2, ..., sN}$, onde $N >= 5$
* Duas ações deterministicas em cada estado (direita ou esquerda) $A(S) = {0, 1}$ onde `0 = esquerda` e `1 = direita`
* Uniform Random Policy $\pi(.|s) = 0.5$ para todo $s$ pertencente a $S$
* Os episodios começam no estado $s3$
* Todos os episodios terminam ao serem atingidos a extremidade esquerda($L$) ou direita($D$)
* $S = {L, s1, s2, s3, ..., sN, D}$
* Os rewards sõ 0 para todas as transições exceto na que leva de $sN$ para $D$, onde o reward é $+1$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
class MyEnv:
    def __init__(self, N = 5, init_state = 3):
        self.N = N #Numero de estados não terminais
        self.init_state = init_state #Estado no qual sera iniciado cada episodio
        self.rewards = np.zeros((self.N + 2)) #Vetor com os rewards de cada estado incluindo os estados terminaais
        self.rewards[-1] = 1 #O reward do ultimo estado é 1
    
    def init(self):
        self.state = self.init_state
        self.terminal = False
        reward = self.rewards[self.state]
        return self.state, reward, self.terminal
        
    def step(self, action):
        if (self.terminal == False):
            if(action == 0): #Se for para a esquerda
                if(self.state == 1): #Se o estado atual for o estado 1
                    self.terminal = True #O episodio terminou pois chegou no estado terminal da esquerda
                else:
                    self.terminal = False
                self.state = self.state - 1
                reward = self.rewards[self.state]

            elif(action == 1):
                if(self.state == self.N):
                    self.terminal = True
                else:
                    self.terminal = False
                self.state = self.state + 1
                reward = self.rewards[self.state]
        else:
            reward = 0
            
        return self.state, reward, self.terminal
            
        
    def getStates(self):
        return np.arange(self.N)
    
    def getActions(self):
        return [0, 1]

In [11]:
class myAgent:
    def __init__(self, pred_type, pol, init_values = 0.5, alpha = 0.1, gamma = 1):
        self.pred_type = pred_type
        self.init_values = init_values
        self.pol = pol
        self.alpha = alpha
        self.gamma = gamma
        
    def train(self, env, n_episodes):
        #Tecnica de prediction para Monte Carlo Method
        if(self.pred_type == "MCM"): 
            #Inicializa o array dos values de cada estado com o numero de estados do evirorment
            self.n_states = len(env.getStates())
            self.V = np.full((self.n_states), self.init_values)
            #Array com os returns de cada estado
            returns = [] 
            #Inicializa os returns com uma lista vazia para cada estado
            for _ in range(self.n_states):
                state_g = []
                returns.append(state_g)
            #Realiza cada episode
            for _ in range(n_episodes):
                #Arrays dos estados por onde passou e as rewards nesses estados
                states = []
                rewards = []
                #Inicializa o envirorment
                state, reward, terminal = env.init()
                #Enquanto não atingir o estado terminal
                while(terminal == False):
                    states.append(state)
                    action = self.argmax(pi[state - 1], 2)
                    state, reward, terminal = env.step(action)
                    rewards.append(reward)
                G = 0
                ind = len(states) - 1
                for rew in reversed(rewards):
                    G = self.gamma*G + rew
                    returns[states[ind] - 1].append(G)
                    self.V[states[ind] - 1] = np.mean(returns[states[ind] - 1])
                    ind -= 1
            return self.V
        elif(self.pred_type == "TD"):
            self.n_states = len(env.getStates())
            self.V = np.full((self.n_states + 2), self.init_values)
            self.V[0] = 0
            self.V[-1] = 0
            for _ in range(n_episodes):
                next_state, reward, terminal = env.init()
                while(terminal == False):
                    state = next_state
                    action = self.argmax(pi[state - 1], 2)
                    next_state, reward, terminal = env.step(action)
                    self.V[state] = self.V[state] + self.alpha*(reward + self.gamma*self.V[next_state] - self.V[state])
            return self.V
    
    
    def argmax(self, q_values, act_num):
        '''Funcao que retorna o indice do valor maximo de um vetor q_values, 
        caso haja mais de um valor maximo entao retorna um deles aletoriamente'''
        top_value = float("-inf") #maior Q(a)
        ties = [] #Acoes que empataram entre o maior valor de Q(a)
        for i in range(act_num):
            if(q_values[i] > top_value):     #Se o Q(a) for maior que o maximo ate o momento:
                top_value = q_values[i]          #Substitui o Q(a) maximo pelo valor do atual
                ties = []                        #Zera o vetor de empates das acoes com o Q(a) maximo
                ties.append(i)                   #Adiciona a acao atual no vetor de empates
            elif(q_values[i] == top_value):  #Se o Q(a) for igual ao maximo ate o momento:
                ties.append(i)                   #Adiciona a ação ao vetor de empates
        return np.random.choice(ties)            #Retorna um valor aleatorio do vetor de empates do Q maximo

In [33]:
n_states = 6
env = MyEnv(N = n_states)
env.init()
num_a = len(env.getActions())
num_s = len(env.getStates())
pi = np.full((num_s, num_a), 0.5)

In [38]:
agent_mcm = myAgent("MCM", pi)
agent_td = myAgent("TD", pi)

V_MCM = agent_mcm.train(env, 3000)
V_TD = agent_td.train(env, 3000)

print("MCM V(S):\t ", V_MCM)
print("TD V(S):\t ", V_TD[1:-1])

MCM V(S):	  [0.13792076 0.28738525 0.4373436  0.57654227 0.7118777  0.85453825]
TD V(S):	  [0.12428888 0.30055886 0.46776297 0.61284068 0.74499033 0.8826055 ]
